In [1]:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext("local","first app")

입력 데이터:

* 다운로드: input.zip
* 설명: 영화DB사이트 (MovieLens)에서 수집된 데이터로서 u.data에는 사용자가 영화에 매긴 평점이 담겨있고 u.item에는 영화마다 제목, 카테고리 등의 정보가 담겨 있다.


입력파일 1 - u.data

* 라인마다 사용자의 영화평점이 기록되어 있으며 4개의 속성 "user id \t item id \t rating \t timestamp"가 탭으로 구분되어 작성되어 있다. (즉, split("\t") 로 자를 수 있음)
* 모두 integer값이다.
* 한 사람이 여러개의 영화에 평점을 매길 수 있으므로 같은 user id로 시작하는 라인이 복수번 나타날수 있다.
* item id는 아래 u.item파일에 있는 영화 아이디 (movie id)를 가리킨다 (즉, foreign key)
* rating은 1에서 5까지 범위이다.


입력파일 2 - u.item

* 라인마다 영화 정보가 기록되어 있으며 "movie id | movie title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation | Children's | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western" 의 속성 들이 파이프라인(|)으로 구분되어 작성되어 있다.
* 그러나 아래 문제를 보면 맨 앞의 두개 속성, movie id와 movie title 만 사용한다.
* movie id는 이 파일 안에서 한번만 나타난다 (즉, primary key)


문제 1.
* 5점을 받은 영화 중에서 알파벳 'T'로 시작하는 영화의 개수는?
* 'T'는 대문자
* 주의: 5점을 받은 영화중 알파벳 'T'로 시작하는 영화를 중복해서 세지 않도록!
* 제한조건: 마지막 action으로 답이 출력되도록 코드를 작성하시오. 즉, 최종 답을 계산하기 위해 pyspark action이 아닌 python 코드를 사용하지 마시오.

문제 2.
* 사용자들에게 1점도 받고 5점도 받은 적이 있는 영화의 제목의 길이 중 가장 긴 영화제목의 길이는 [공백 1개]이다.
* 동일 영화가 어떤 사용자에게는 1점, 어떤 사용자에게는 5점을 받은 경우를 말함
* 힌트: cogroup을 쓸 수도 있고 groupByKey를 쓸 수도 있음.
* 제한조건: 마지막 action으로 답이 출력되도록 코드를 작성하시오. 즉, 최종 답을 계산하기 위해 pyspark action이 아닌 python 코드를 사용하지 마시오.

# 문제1 

In [53]:
data1 = sc.textFile("/content/drive/MyDrive/Colab Notebooks/데이터분석을위한프로그래밍/u.data",1)
data2 = sc.textFile("/content/drive/MyDrive/Colab Notebooks/데이터분석을위한프로그래밍/u.item",1)

In [12]:
data1.take(5)
# user_id, item_id, rating, timestamp

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596']

In [47]:
data1.map(lambda x :x.split('\t')).map(lambda x:(x[:3])).filter(lambda x: x[2]=='5').map(lambda x:x[1]).distinct().map(lambda x:(x,1))\
.take(3)
#map(lambda x:eval(x)).collect()
#
#collect()
#filter(lambda x: x[2]==5).collect()



[('465', 1), ('1014', 1), ('222', 1)]

In [13]:
data2.take(5)
#movie id | movie title~

['1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0',
 '2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0',
 '3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0',
 '4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0',
 '5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0']

In [49]:
data2.map(lambda x :x.split("|")).map(lambda x:(x[0],x[1])).filter(lambda x: x[1][0]=='T').take(3)

[('1', 'Toy Story (1995)'),
 ('7', 'Twelve Monkeys (1995)'),
 ('23', 'Taxi Driver (1976)')]

In [54]:
data1 = data1.map(lambda x :x.split('\t')).map(lambda x:(x[:3])).filter(lambda x: x[2]=='5').map(lambda x:x[1]).distinct().map(lambda x:(x,1))
data2=data2.map(lambda x :x.split("|")).map(lambda x:(x[0],x[1])).filter(lambda x: x[1][0]=='T')

data1.join(data2).count()

75

# 문제 2 

In [81]:
data1 = sc.textFile("/content/drive/MyDrive/Colab Notebooks/데이터분석을위한프로그래밍/u.data",1)
data2 = sc.textFile("/content/drive/MyDrive/Colab Notebooks/데이터분석을위한프로그래밍/u.item",1)

In [68]:
data1.map(lambda x :x.split('\t')).map(lambda x:(x[1],x[2])).filter(lambda x: x[1]=='1' or x[1]=='5').distinct().groupByKey()\
.mapValues(list).filter(lambda x: len(x[1])==2).take(3)

[('346', ['1', '5']), ('465', ['5', '1']), ('1014', ['5', '1'])]

In [70]:
data2.map(lambda x :x.split("|")).map(lambda x:(x[0],x[1])).take(3)

[('1', 'Toy Story (1995)'),
 ('2', 'GoldenEye (1995)'),
 ('3', 'Four Rooms (1995)')]

In [83]:
data1 = sc.textFile("/content/drive/MyDrive/Colab Notebooks/데이터분석을위한프로그래밍/u.data",1)
data2 = sc.textFile("/content/drive/MyDrive/Colab Notebooks/데이터분석을위한프로그래밍/u.item",1)
data1=data1.map(lambda x :x.split('\t')).map(lambda x:(x[1],x[2])).filter(lambda x: x[1]=='1' or x[1]=='5').distinct().groupByKey()\
.mapValues(list).filter(lambda x: len(x[1])==2)
data2 =data2.map(lambda x :x.split("|")).map(lambda x:(x[0],x[1]))
#data1.join(data2).take(3)
data1.join(data2).map(lambda x: len(x[1][-1])).max()

81